<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/nn_tensorflow_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
t = [1, 2, 3, 4, 5]

In [ ]:
t = tf.convert_to_tensor(t)

In [ ]:
t

In [ ]:
type(t)

In [ ]:
t.shape

In [ ]:
t2 = tf.convert_to_tensor([1,2, 3, 4, 5])

In [ ]:
t + t2

In [ ]:
t2.numpy()

In [ ]:
np.max(t)

In [ ]:
tf.reduce_max(t, axis=0, )

In [ ]:
a = np.array([1, 2, 3, 4, 5])

In [ ]:
a[0] = 0

In [ ]:
a

In [ ]:
a_tf = tf.convert_to_tensor(a)

In [ ]:
a_tf[0] = 10

In [ ]:
a_tf = tf.Variable(initial_value=a, dtype=tf.float32)

In [ ]:
a_tf.assign([10, 11, 12, 13, 14])

In [ ]:
a_tf

In [ ]:
a_tf[0].assign(2)

In [ ]:
a_tf

In [ ]:
t

In [ ]:
tf.reduce_sum(a_tf * tf.cast(t, tf.float32))

In [ ]:
t = tf.cast(t, tf.float32)
with tf.GradientTape() as tape:
    # tape.watch(t)
    scalar_product = tf.reduce_sum(a_tf * t)

grad = tape.gradient(scalar_product, a_tf)

In [ ]:
grad

In [ ]:
import tensorflow as tf

In [ ]:
tf.convert_to_tensor(np.array([1., 2, 3, 4, 5]), dtype=tf.float32)

In [ ]:
type(_)

In [ ]:
var = tf.Variable(initial_value=[1, 2, 3], dtype=tf.float32, name='a')

In [ ]:
var

In [ ]:
var.assign([2, 3, 4])

In [ ]:
var.assign_add([1, 1, 1])

In [ ]:
var[0].assign(6)

In [ ]:
var.trainable

In [ ]:
t = tf.convert_to_tensor([1, 2, 3], dtype=tf.float32)

In [ ]:
with tf.GradientTape() as tape:
    sum_ = tf.reduce_sum(var*t)

grad = tape.gradient(sum_, var)

In [ ]:
grad

In [ ]:
with open('filename', 'w'):
    # scrittura sul file

In [ ]:
class Callable:
    def __call__(self):
        print('Sono chiamabile')

In [ ]:
c = Callable()

In [ ]:
c()

In [ ]:
class ContextManager:
    def __enter__(self):
        print('inizializzazione')

    def __exit__(self, exception_type, exception_value, traceback):
        print('chiusura')

In [ ]:
import time

with ContextManager():
    time.sleep(5)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
X = np.random.rand(10_000, 2)
y = 2*X[:, 0] + 3*X[:, 1] - 1

In [ ]:
import tensorflow.keras.datasets as datasets
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train = tf.cast(X_train, dtype=tf.float32) / 255
X_test = tf.cast(X_test, dtype=tf.float32) / 255

In [ ]:
class Mlp:
    def __init__(self, units_list, with_softmax=False):
        '''
        args
        ----

        units_list: lista con il numero di unità per layer
                    [784, 100, 100, 10]
        with_softmax: se l'ultimo starto è seguito da una softmax
        '''
        self.Ws = []
        self.bs = []
        self.with_softmax = with_softmax

        # creazione dei pesi
        for in_units, out_units in zip(units_list[:-1], units_list[1:]):
            # (784, 100)
            W = tf.Variable(initial_value=np.random.normal(size=(in_units, out_units)), shape=[in_units, out_units], dtype=tf.float32)
            b = tf.Variable(initial_value=np.zeros(shape=(out_units,)), shape=[out_units], dtype=tf.float32)
            self.Ws.append(W)
            self.bs.append(b)

    def __call__(self, x):
        '''
        args
        ----
        x: x.shape==(n_batch, n_features)

        ret
        ---
        y: y.shape==(n_batch, n_out)
        '''
        *W_hidden, W_last = self.Ws
        *b_hidden, b_last = self.bs
        # ciclo sui layer nascosti
        for W, b in zip(W_hidden, b_hidden):
            # W.shape = (n_in, n_out)
            # x.shape = (n_batch, n_in)
            x = x@W + b
            
            # applica sigmoide
            x = tf.math.sigmoid(x)

        # ultimo layer
        y = x@W_last + b_last

        if self.with_softmax:
            y = tf.math.softmax(y)

        return y

In [ ]:
mlp = Mlp(units_list=[784, 100, 100, 10], with_softmax=True)

In [ ]:
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

In [ ]:
from tqdm import tqdm

epochs = 10
alpha = 1e-3
batch_size = 32
# regressione lineare

n = 60_000

pbar = tqdm()
for epoch in range(epochs):
    # X @ theta(vett colonna) + b = y_pred
    for idx in range(0, n, batch_size):
        # estrazione del batch
        # x_b.shape = (batch_size, 28, 28), y_b.shape = (batch_size,)
        x_b, y_b = X_train[idx:idx+batch_size], y_train[idx:idx+batch_size]

        with tf.GradientTape() as tape:
            # (batch_size, 28, 28) -> (batch_size, 784)
            x_b_reshaped = tf.reshape(x_b, shape=[batch_size, 28*28])

            # y_pred.shape = (batch_size, 10)
            y_pred = mlp(x_b_reshaped)
            cce = tf.keras.losses.CategoricalCrossentropy()
            # la loss media sugli elementi del batch
            loss = cce(y_b, y_pred)
        pbar.set_description(f'loss={loss.numpy().item():2f}')
        # Ws_grad = [W1_grad (784, 100), W2_grad (100, 100), W3_grad (100, 10)]
        # bs_grad = [b1_grad (100), b2_grad (100), b3_grad (10)]
        Ws_grad, bs_grad = tape.gradient(loss, [mlp.Ws, mlp.bs])
        for W, W_grad, b, b_grad in zip(mlp.Ws, Ws_grad, mlp.bs, bs_grad):
            # step di gradient descent
            # W = W - alpha*W_grad
            W.assign_sub(alpha*W_grad)
            b.assign_sub(alpha*b_grad)


In [ ]:
for idx in range(0, 1000, 32):
    print(idx)

In [ ]:
theta1 = [theta[0] for theta in thetas]
theta2 = [theta[1] for theta in thetas]
plt.plot(theta1, theta2)

In [ ]:
b

In [ ]:
theta